In [ ]:
%reset -f
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import keras
import os
import csv
import os.path


1. Importazione di keras-retinanet dalla repository di GitHub attraverso il comando **git clone**

In [ ]:
!git clone https://github.com/fizyr/keras-retinanet.git

2. Installazione di keras-retinanet

In [ ]:
%cd keras-retinanet/

!pip install .

Per utilizzare la rete di backbone EfficientNetB2 conviene utilizzare una versione precedente di keras e tensorflow

In [ ]:
!pip install keras==2.4 --user
!pip install tensorflow==2.3.0 --user

In [ ]:
!python setup.py build_ext --inplace

**PREPARAZIONE DEI DATI**

3. Download del dataset WiderPerson in formato .zip, utilizzando il comando **wget** ed estrazione tramite **unzip**

In [ ]:
!wget --user=corso --password=p2021corso http://www.grip.unina.it/download/corso/WiderPerson.zip

In [ ]:
!unzip WiderPerson.zip

4. Creazione dei 3 file .csv(*test_set.csv, training_set.csv, validation_set.csv*)

In [ ]:

count=0
for cartella, sottocartella, files in os.walk('WiderPerson/Images/'):
    for file in files:
      nome=os.path.basename(file)
      
      path='WiderPerson/Annotations/'+nome+'.txt'
      if(os.path.exists(path)):
        count=count+1
        with open(path) as input_file:
            num=input_file.readline()
            lines =input_file.readlines()
            for line in lines:
               newLine =line.strip().split()
               newLine.append('WiderPerson/Images/'+nome)
               tmp=newLine[0]
               newLine[0]=newLine[5]
               newLine[5]=tmp
               if(newLine[5]=='1'):
                 newLine[5]='pedestrians'
               

               if(count<=1000):
                 if(newLine[5]=='pedestrians'):
                   with open('test_set.csv', 'a', newline='') as test_file:                    
                     file_writer = csv.writer(test_file)
                     file_writer.writerow(newLine)
               elif(count<=7000):
                 if(newLine[5]=='pedestrians'):
                   with open('training_set.csv', 'a', newline='') as training_file:                    
                     file_writer = csv.writer(training_file)
                     file_writer.writerow(newLine)
               else:
                 if(newLine[5]=='pedestrians'):
                   with open('validation_set.csv', 'a', newline='') as validation_file:                    
                     file_writer = csv.writer(validation_file)
                     file_writer.writerow(newLine)

5. Creazione file di mapping, contenente nome ed id delle classi da individuare(*mapping.csv*)

In [ ]:
 with open('mapping.csv', 'w', newline='') as map_file:                    
        file_writer = csv.writer(map_file)
        file_writer.writerow(['pedestrians','0'])
       


**FASE DI TRAINING**

In [ ]:
!pip install efficientnet

6. Avvio dell' addestramento della rete utilizzando lo script *train.py* con annessi parametri

In [ ]:
import keras_retinanet
!python /content/keras-retinanet/keras_retinanet/bin/train.py --backbone EfficientNetB2 --freeze-backbone --steps 200 --batch-size 4 --epochs 30 --lr 1e-4 csv training_set.csv mapping.csv


**FASE DI TESTING**

7. Avvio del testing sul file *test_set.csv* con l' utilizzo dello script *evaluate.py*(per convertire il modello in un inference model si utilizza il comando **--convert-model**)

In [ ]:
!python keras_retinanet/bin/evaluate.py --backbone EfficientNetB2 csv test_set.csv mapping.csv ./snapshots/EfficientNetB2_csv_30.h5 --convert-model

8. Test finale per visualizzare i risultati su immagini reali

In [ ]:
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu
import cv2
import time

from efficientnet.tfkeras import EfficientNetB2


model_path = os.path.join('', 'snapshots', 'EfficientNetB2_csv_30.h5')

backbone = models.backbone('EfficientNetB2')
# load retinanet model
model =models.load_model(model_path, backbone_name='EfficientNetB2')
model = models.convert_model(model)

labels_to_names = {0: 'Pedestrian'}
# load image
image = read_image_bgr('WiderPerson/Images/004167.jpg')
#WiderPerson/Images/014562.jpg,61,123,151,343,pedestrians
# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

# preprocess image for network
image = preprocess_image(image, mode='tf')
image, scale = resize_image(image)

# process image
start = time.time()
boxes, scores, labels= model.predict_on_batch(np.expand_dims(image, axis=0))
print("processing time: ", time.time() - start)

# correct for image scale
boxes /= scale

# visualize detections
for box, score, label in zip(boxes[0], scores[0], labels[0]):
    # scores are sorted so we can break
    if score < 0.4:
        break
        
    color = label_color(label)
    
    b = box.astype(int)
    draw_box(draw, b, color=color)
    #print(b,label,color)
    caption = "{} {:.3f}".format(labels_to_names[label], score)
    draw_caption(draw, b, caption)
    
#draw_box(draw, [61,123,151,343], color=[255,0,0]) 
plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(draw)
plt.show()